In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import csv
import seaborn as sns
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

In [ ]:
train = pd.read_csv("../input/stumbleupon/train.tsv", delimiter="\t")
test = pd.read_csv("../input/stumbleupon/test.tsv", delimiter="\t")
sample = pd.read_csv("../input/stumbleupon/sampleSubmission.csv")

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
test.info()

In [ ]:
train.head(2)

In [ ]:
test.head(2)

In [ ]:
sample.head(2)

In [ ]:
train.columns

In [ ]:
train.head(1).loc[0]

In [ ]:
list(zip(train.columns, train.dtypes, train.nunique()))

In [ ]:
"""
object columns

url
boilerplate
alchemy_category        14
alchemy_category_score
is_news                 2
news_front_page         3
"""

In [ ]:
train.head(2).url

In [ ]:
train.head(2).boilerplate

In [ ]:
train.head(2).alchemy_category

In [ ]:
train.head(2).alchemy_category_score

In [ ]:
train.head(2).is_news

In [ ]:
train.head(2).news_front_page

In [ ]:
#url
#boilerplate
#alchemy_category_score

In [ ]:
train.head(2).boilerplate[0].split(":")[1]

In [ ]:
train["boilerplate_str"]=train.boilerplate.apply(lambda x:x.split(":")[1])
test["boilerplate_str"]=test.boilerplate.apply(lambda x:x.split(":")[1])

In [ ]:
train.drop('boilerplate', inplace=True, axis=1)
test.drop('boilerplate', inplace=True, axis=1)

In [ ]:
train.shape, test.shape

In [ ]:
train.head(2)

In [ ]:
train.alchemy_category_score[train.alchemy_category_score=="?"] =0

In [ ]:
test.alchemy_category_score[test.alchemy_category_score=="?"] =0

In [ ]:
train.alchemy_category_score

In [ ]:
train.alchemy_category_score= train.alchemy_category_score.apply(lambda x:float(x))
test.alchemy_category_score=test.alchemy_category_score.apply(lambda x:float(x))

In [ ]:
train.alchemy_category_score

In [ ]:
list(zip(train.columns, train.dtypes, train.nunique()))

In [ ]:
train.select_dtypes(['object'])

In [ ]:
y = train.label
train.drop('label', axis=1, inplace=True)

In [ ]:
train.shape, test.shape

In [ ]:
"""
now we have three text categories
alchemy_category        14
is_news                 2
news_front_page         3

"""
dummies_col = ['alchemy_category','is_news','news_front_page']

In [ ]:
train[dummies_col]

In [ ]:
train['flage'] ='train'
test['flage'] ='test'

In [ ]:
train.shape, test.shape

In [ ]:
dataset_1 = pd.concat([train, test], axis= 0)

In [ ]:
pd.get_dummies(dataset_1['alchemy_category'],drop_first=True)

In [ ]:
dataset_1.shape

In [ ]:
dataset_1.head()

In [ ]:
dataset_1=pd.get_dummies(dataset_1, columns = dummies_col)

In [ ]:
dataset_1.shape

In [ ]:
dataset_1.head(2)

In [ ]:
dataset_1.columns

In [ ]:
train.shape, test.shape

In [ ]:
train = dataset_1[dataset_1.flage=='train']
test = dataset_1[dataset_1.flage=='test']

In [ ]:
train.shape, test.shape

In [ ]:
train.select_dtypes(['object'])

In [ ]:
train.drop('flage', axis=1, inplace=True)
test.drop('flage', axis=1 , inplace=True)

In [ ]:
train.shape, test.shape

In [ ]:
train['label'] = y

In [ ]:
train.shape, test.shape

## So no we have two columns for text classification Let's use zone indexing

In [ ]:
zone1_train= train[['urlid','url']]
zone1_test= test[['urlid','url']]

In [ ]:
zone1_train.url[0]

In [ ]:
zone1_train.url[0].replace("/"," ")

In [ ]:
import string
string.punctuation

In [ ]:
from nltk.corpus import stopwords
#stopwords.words('english')

In [ ]:
def text_process(mess):
    mess=mess.replace("/"," ")
    mess = mess.replace("-" ," ")
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = "".join(nopunc)
    return [porter.stem(word) for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
text_process(zone1_train.url[0])

In [ ]:
zone1_train.url[0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer = text_process)
bow_transformer.fit(zone1_train['url'])

In [ ]:
#bow_transformer.vocabulary_

In [ ]:
messages_bow_train = bow_transformer.transform(zone1_train['url'])
messages_bow_test = bow_transformer.transform(zone1_test['url'])

In [ ]:
(messages_bow_train.todense() != 0).sum()

In [ ]:
messages_bow_train.shape

In [ ]:
#

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(messages_bow_train)
tfidf4_train = tfidf_transformer.transform(messages_bow_train)
tfidf4_test = tfidf_transformer.transform(messages_bow_test)

In [ ]:
(tfidf4_train.todense() != 0).sum()

In [ ]:
d1 = pd.DataFrame(tfidf4_train.todense())
d1

In [ ]:
d2 = pd.DataFrame(tfidf4_test.todense())
d2

In [ ]:
train=pd.concat([train, d1], axis= 1)
test=pd.concat([test, d2], axis= 1)

In [ ]:
train.shape, test.shape

# Zone2

In [ ]:
zone2_train= train[['urlid','boilerplate_str']]
zone2_test= test[['urlid','boilerplate_str']]

In [ ]:
def text_process2(mess):
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = "".join(nopunc)
    return [porter.stem(word) for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
text_process2(zone2_train.boilerplate_str[0])

In [ ]:
zone2_train.boilerplate_str[0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer = text_process2)
bow_transformer.fit(zone2_train['boilerplate_str'])

In [ ]:
messages_bow_train = bow_transformer.transform(zone2_train['boilerplate_str'])
messages_bow_test = bow_transformer.transform(zone2_test['boilerplate_str'])

In [ ]:
(messages_bow_train.todense() != 0).sum()

In [ ]:
tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(messages_bow_train)
tfidf5_train = tfidf_transformer.transform(messages_bow_train)
tfidf5_test = tfidf_transformer.transform(messages_bow_test)

In [ ]:
(tfidf5_train.todense() != 0).sum()

In [ ]:
d3 = pd.DataFrame(tfidf5_train.todense())
d4 = pd.DataFrame(tfidf5_test.todense())

In [ ]:
train.shape, test.shape

In [ ]:
train.head(2)

In [ ]:
test.head(2)

In [ ]:
list(train.columns)[-2:],list(test.columns)[-2:]

In [ ]:
train=pd.concat([train, d3], axis= 1)
test=pd.concat([test, d4], axis= 1)

In [ ]:
train.shape, test.shape

In [ ]:
train.select_dtypes(['object'])

In [ ]:
train.drop('url', axis=1, inplace=True)
test.drop("url", axis=1, inplace=True)

train.drop('boilerplate_str', axis=1, inplace=True)
test.drop("boilerplate_str", axis=1, inplace=True)

In [ ]:
train.shape, test.shape

## All set let's use SVM

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split

In [ ]:
y = train.label
X = train.drop("label" , axis=1)

## PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components= 2)
pca.fit(X)
X = pca.transform(X)
test1 = pca.transform(test)

In [ ]:
X.shape, y.shape, test1.shape

## PCA close

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report


## default SVM

In [ ]:
model = svm.SVC(gamma='scale',kernel='sigmoid')
model.fit(X_train, y_train)
train_pred=model.predict(X_train)
test_pred = model.predict(X_test)
print("Evaluate for train")
print(confusion_matrix(y_train, train_pred))
print(classification_report(y_train, train_pred))
print("-"*40)
print("Evaluate for test")
print(confusion_matrix(y_test, test_pred))
print(classification_report(y_test, test_pred))

In [ ]:
sns.heatmap(confusion_matrix(y_test, test_pred))

## gamma='scale',kernel='rbf'

In [ ]:
model = svm.SVC(gamma='scale',kernel='rbf')
model.fit(X_train, y_train)
train_pred=model.predict(X_train)
test_pred = model.predict(X_test)
print("Evaluate for train")
print(confusion_matrix(y_train, train_pred))
print(classification_report(y_train, train_pred))
print("-"*40)
print("Evaluate for test")
print(confusion_matrix(y_test, test_pred))
print(classification_report(y_test, test_pred))

In [ ]:
sns.heatmap(confusion_matrix(y_test, test_pred))

## gamma='scale', kernel='linear'

In [ ]:
# model = svm.SVC(gamma='scale', kernel='linear')
# model.fit(X_train, y_train)
# train_pred=model.predict(X_train)
# test_pred = model.predict(X_test)
# print("Evaluate for train")
# print(confusion_matrix(y_train, train_pred))
# print(classification_report(y_train, train_pred))
# print("-"*40)
# print("Evaluate for test")
# print(confusion_matrix(y_test, test_pred))
# print(classification_report(y_test, test_pred))

In [ ]:
sample.label=model.predict(test1)

In [ ]:
sample.to_csv("sub1.csv",index=False)